In [3]:
import pandas as pd
years = [18, 19, 20, 21, 22, 23, 24, 25]
headers = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build_flag",
    "tenure_type",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]
drop = [
    "transaction_id",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "ppd_category_type",
    "record_status"
]
dfs = {}
for year in years:
    url = f"http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-20{year}.csv"
    df = pd.read_csv(url)
    df.columns = headers
    df = df.drop(columns=drop)
    dfs[f"df20{year}"] = df
    print(f"{year} done")
dfs

18 done
19 done
20 done
21 done
22 done


C:\Users\lenovo\AppData\Local\Temp\ipykernel_7788\2815900718.py:33: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


23 done
24 done
25 done


{'df2018':           price  date_of_transfer  postcode property_type new_build_flag  \
 0        253500  2018-09-24 00:00    M6 8GQ             D              N   
 1        231950  2018-09-28 00:00   WA3 2UE             D              Y   
 2        112500  2018-08-29 00:00   OL6 6RJ             S              N   
 3        184995  2018-06-15 00:00   M46 0TW             S              Y   
 4        214995  2018-09-28 00:00   M28 3XS             D              Y   
 ...         ...               ...       ...           ...            ...   
 1037248  100000  2018-11-16 00:00   DH4 7HW             T              N   
 1037249  334950  2018-11-30 00:00  NE31 1BF             D              Y   
 1037250  106010  2018-12-12 00:00   NE2 1NY             F              N   
 1037251  144995  2018-07-26 00:00  NE27 0JA             T              Y   
 1037252  334950  2018-07-13 00:00  NE15 9BW             D              Y   
 
         tenure_type            town_city             district  

In [4]:
df_full = pd.concat(dfs)

In [5]:
df_full['Price (Thousands)'] = df_full['price'] / 1000
df_full.drop(columns=['price'], inplace=True)
df_full.describe()

,Price (Thousands)
count,7.219742e+06
mean,3.818475e+02
std,1.711452e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [6]:
df_full.isna().sum()

date_of_transfer         0
postcode             24503
property_type            0
new_build_flag           0
tenure_type              0
town_city                0
district                 0
county                   0
Price (Thousands)        0
dtype: int64

In [22]:
df_clean = df_full.dropna(axis=1)
df_clean.describe()

,Price (Thousands)
count,7.219742e+06
mean,3.818475e+02
std,1.711452e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [23]:
df_binary = pd.get_dummies(df_clean, dtype=int, columns=['tenure_type','new_build_flag', 'property_type'])
df_binary.rename(columns={'date_of_transfer':'Transfer Date','town_city':'Town/City','district':'District','county':'County','tenure_type_F':'Freehold Tenure','tenure_type_L':'Leasehold Tenure','new_build_flag_N':'Old Build','new_build_flag_Y':'New Build','property_type_D':'Detached', 'property_type_F':'Flat', 'property_type_O':'Other Property Type','property_type_S':'Semi-detached','property_type_T':'Terraced'}, inplace=True)
df_binary.head()

Transfer Date          Town/City  District              County  \
df2018 0  2018-09-24 00:00            SALFORD   SALFORD  GREATER MANCHESTER   
       1  2018-09-28 00:00         WARRINGTON     WIGAN  GREATER MANCHESTER   
       2  2018-08-29 00:00  ASHTON-UNDER-LYNE  TAMESIDE  GREATER MANCHESTER   
       3  2018-06-15 00:00         MANCHESTER     WIGAN  GREATER MANCHESTER   
       4  2018-09-28 00:00         MANCHESTER   SALFORD  GREATER MANCHESTER   

          Price (Thousands)  Freehold Tenure  Leasehold Tenure  Old Build  \
df2018 0            253.500                1                 0          1   
       1            231.950                1                 0          0   
       2            112.500                1                 0          1   
       3            184.995                1                 0          0   
       4            214.995                0                 1          0   

          New Build  Detached  Flat  Other Property Type  Semi-detached  \
df2018 0          0         1     0                    0              0   
       1          1         1     0                    0              0   
       2          0         0     0                    0              1   
       3          1         0     0                    0              1   
       4          1         1     0                    0              0   

          Terraced  
df2018 0         0  
       1         0  
       2         0  
       3         0  
       4         0

In [24]:
df_binary['Transfer Date'] = df_binary['Transfer Date'].astype(str).str[:10]
df_binary.tail()

Transfer Date    Town/City                District  \
df2025 252142    2025-01-17  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252143    2025-01-24    KETTERING  NORTH NORTHAMPTONSHIRE   
       252144    2025-01-14      RUSHDEN  NORTH NORTHAMPTONSHIRE   
       252145    2025-01-10  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252146    2025-01-27    KETTERING  NORTH NORTHAMPTONSHIRE   

                               County  Price (Thousands)  Freehold Tenure  \
df2025 252142   WEST NORTHAMPTONSHIRE              98.00                0   
       252143  NORTH NORTHAMPTONSHIRE             120.00                0   
       252144  NORTH NORTHAMPTONSHIRE             604.50                1   
       252145   WEST NORTHAMPTONSHIRE             133.65                0   
       252146  NORTH NORTHAMPTONSHIRE             332.00                1   

               Leasehold Tenure  Old Build  New Build  Detached  Flat  \
df2025 252142                 1          1          0         0     1   
       252143                 1          1          0         0     1   
       252144                 0          1          0         0     0   
       252145                 1          1          0         0     1   
       252146                 0          1          0         1     0   

               Other Property Type  Semi-detached  Terraced  
df2025 252142                    0              0         0  
       252143                    0              0         0  
       252144                    1              0         0  
       252145                    0              0         0  
       252146                    0              0         0

In [25]:
lower = df_binary['Price (Thousands)'].quantile(0.005)
upper = df_binary['Price (Thousands)'].quantile(0.995)
df_binary = df_binary[(df_binary['Price (Thousands)'] >= lower) & (df_binary['Price (Thousands)'] <= upper)]
df_binary.describe()

,Price (Thousands),Freehold Tenure,Leasehold Tenure,Old Build,New Build,Detached,Flat,Other Property Type,Semi-detached,Terraced
count,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06,7.147613e+06
mean,3.298730e+02,7.680538e-01,2.319462e-01,8.883434e-01,1.116566e-01,2.332777e-01,1.775309e-01,4.977704e-02,2.673146e-01,2.720998e-01
std,2.809675e+02,4.220749e-01,4.220749e-01,3.149435e-01,3.149435e-01,4.229176e-01,3.821174e-01,2.174840e-01,4.425579e-01,4.450410e-01
min,1.289500e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.650000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.600000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.999500e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,3.075000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [27]:
df_binary.to_csv('ETL_Final.csv', index=False)